# 데이터 전처리용 코드
1. 추출형 요약용 데이터: tf-idf나 SentenceBERT를 통해서 얻은 문장 유사도 데이터 파일을 기반으로 KoBERTSum 등의 요약 모델에서 사용 가능한 extractive summarization용 데이터 파일을 생성합니다.
2. 추상형 요약용 데이터: 법원 원데이터를 기반으로 준비서면/소장 텍스트를 전문으로, 판결문 텍스트를 요약문으로 하는 abstractive summarization용 데이터 파일을 생성합니다. 

In [ ]:
import re
import csv
import json
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

_whitespace = re.compile(r'\s+')
_valid_chars = re.compile(r"[^ㄱ-ㅣ가-힣a-zA-Z0-9\s;:,.!?¡¿—-…«»'\"“”~(){}[\]\-]")  # 남겨둘 한글, 영문, 주요 특수문자 설정

In [ ]:
def collapse_whitespace(text):  # 중복된 공백 하나로 압축
    return re.sub(_whitespace, ' ', text)


def cleaner(text):  # 한글 + 영문 + 주요 특수문자 + 공백만 남기기
    clean_text = re.sub(_valid_chars, '', text)
    return clean_text


def remover(text):  # 이상하거나 부자연스러운 표기 교정
    text = text.replace('(idnum)', '')
    text = text.replace('(phonenum)', '')
    text = text.replace('idnum', '')
    text = text.replace('phonenum', '')
    text = text.replace('지 급 명 령 신 청 서', '지급명령신청서')
    text = text.replace('청 구 취 지', '청구취지')
    text = text.replace('준 비 서 면', '준비서면')
    text = text.replace('참 고 서 면', '참고서면')
    text = text.replace('입 증 방 법', '입증방법')
    text = text.replace('참 고 자 료', '참고자료')
    text = text.replace('답 변 서', '답변서')
    text = text.replace('다 음 ', '다음 ')
    text = text.replace('결 론', '결론')
    text = text.replace(' 원 고 ', ' 원고 ')
    text = text.replace(' 피 고 ', ' 피고 ')
    text = text.replace(' 사 건 ', '사건')

    return text

In [ ]:
brief_lens = list()
judgement_lens = list()

with open('./Sentence-Embedding-Is-All-You-Need/data.csv', 'r') as f:
    reader = csv.reader(f)

    for row in tqdm(reader):
        brief_lens.append(len(row[3]))
        judgement_lens.append(len(row[5]))

In [ ]:
plt.hist(brief_lens, bins=100, label='brief')
plt.hist(judgement_lens, bins=100, label='judgement')
plt.legend(loc='upper right')
plt.axis([0, 2000, 0, 80000])
plt.show()

In [ ]:
# Convert to Ext data
# 매커니즘: 준비서면 문장별로 |를 경계로 묶고, 판결문과 유사한 내용을 담은 문장을 요약문에 포함시킴. 요약문에 포함된 문장 인덱스를 모아서 target으로 사용함.
# 조건1: 준비서면쪽 문장 길이 제한 - 하한 상한
# 조건2: 판결문쪽 문장 길이 - 하한 상한
# 조건3: 유사도 제한 - 하한 상한
# 결과물: ext_data.csv 확인

# hyper-parameter tuning 필요함
min_brief_len = 15
max_brief_len = 500

min_judgement_len = 15
max_judgement_len = 750

min_similarity = 0.75
max_similarity = 0.95

data_list = list()

with open('./Sentence-Embedding-Is-All-You-Need/data.csv', 'r', encoding='utf-8-sig', newline='') as f:
    reader = csv.reader(f)

    current_case = None
    current_document = None
    target_judgement = None
    sentences = list()
    target_index_list = list()
    sentence_idx = 0

    for idx, row in tqdm(enumerate(reader), total=82192):
        if idx != 0:
            if current_document != row[1]:
                if len(target_index_list):
                    summary = ''
                    for i in target_index_list:
                        summary += sentences[i]
                    data_list.append([current_case, current_document, target_judgement, '|'.join(sentences), target_index_list, summary])

                current_case =  row[0] 
                current_document = row[1]
                target_judgement = row[2]
                sentences = list()
                target_index_list = list()
                sentence_idx = 0

            brief_len = len(row[3])
            judgement_len = len(row[5])
            similarity = float(row[4])

            # 주요 문장 필터링
            if brief_len > min_brief_len and brief_len < max_brief_len and judgement_len > min_judgement_len and judgement_len < max_judgement_len and similarity > min_similarity and similarity < max_similarity:
                target_index_list.append(sentence_idx)
            
            sentences.append(row[3])
            sentence_idx += 1
            
    if len(target_index_list):
        summary = ''
        for i in target_index_list:
            summary += sentences[i]
        data_list.append([current_case, current_document, target_judgement, '|'.join(sentences), target_index_list, summary])


with open('./Sentence-Embedding-Is-All-You-Need/ext_data.csv', 'w', encoding='utf-8-sig', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(['case_number', 'brief_edms_id', 'judgement_edms_id', 'brief_sentence', 'sentence_index', 'summary'])
    
    for data in tqdm(data_list, total=len(data_list)):
        wr.writerow(data)


In [ ]:
# Convert to Ext data (Dacon 데이터 형식)
# 매커니즘: 준비서면 문장별로 모아 리스트를 만들고, 판결문과 유사한 내용을 담은 문장을 요약문으로 상정함. 요약문에 포함된 문장 인덱스를 모아서 target으로 사용함.
# 조건1: 준비서면쪽 문장 길이 제한 - 하한 상한
# 조건2: 판결문쪽 문장 길이 - 하한 상한
# 조건3: 유사도 제한 - 하한 상한
# 결과물: train.jsonl과 extractive_test_v2.jsonl 확인

# hyper-parameter tuning 필요함
min_brief_len = 15
max_brief_len = 500

min_judgement_len = 15
max_judgement_len = 750

min_similarity = 0.75
max_similarity = 0.95

data_list = list()

with open('./Sentence-Embedding-Is-All-You-Need/data.csv', 'r', encoding='utf-8-sig', newline='') as f:
    reader = csv.reader(f)

    current_case = None
    current_document = None
    target_judgement = None
    sentences = list()
    target_index_list = list()
    sentence_idx = 0

    for idx, row in tqdm(enumerate(reader), total=82192):
        if idx != 0:
            if current_document != row[1]:
                if len(target_index_list):
                    summary = ''
                    for i in target_index_list:
                        summary += sentences[i]
                    data_list.append([current_case, current_document, sentences, summary, target_index_list])

                current_case =  row[0] 
                current_document = row[1]
                target_judgement = row[2]
                sentences = list()
                target_index_list = list()
                sentence_idx = 0

            brief_len = len(row[3])
            judgement_len = len(row[5])
            similarity = float(row[4])
            if brief_len > min_brief_len and brief_len < max_brief_len and judgement_len > min_judgement_len and judgement_len < max_judgement_len and similarity > min_similarity and similarity < max_similarity:
                target_index_list.append(sentence_idx)
            
            sentences.append(row[3])
            sentence_idx += 1
            
    if len(target_index_list):
        summary = ''
        for i in target_index_list:
            summary += sentences[i]
        data_list.append([current_case, current_document, sentences, summary, target_index_list])

with open("./Sentence-Embedding-Is-All-You-Need/train.jsonl" , encoding= "utf-8-sig",mode="w") as f: 
    for data in tqdm(data_list[:2400], total=len(data_list[:2400])):
        line = {"media":data[0],"id":data[1],"article_original":data[2],"abstractive":data[3],"extractive":data[4]}
        f.write(json.dumps(line) + "\n")
        
with open("./Sentence-Embedding-Is-All-You-Need/extractive_test_v2.jsonl" , encoding= "utf-8-sig",mode="w") as f: 
    for data in tqdm(data_list[2400:], total=len(data_list[2400:])):
        line = {"media":data[0],"id":data[1],"article_original":data[2],"abstractive":data[3],"extractive":data[4]}
        f.write(json.dumps(line) + "\n")


In [ ]:
# Pandas를 통해 jsonl의 데이터 확인
with open('./Sentence-Embedding-Is-All-You-Need/ext_data_test.jsonl', 'r', encoding='utf-8-sig') as json_file:
    json_list = list(json_file)

trains = []
for json_str in json_list:
    line = json.loads(json_str)
    trains.append(line)

train_df = pd.DataFrame(trains[0])
train_df.head()

In [ ]:
# Convert to Abs data
# 매커니즘: 준비서면/소장-판결문이 매칭된 데이터셋 제작.

data_list = list()

with open('./Sentence-Embedding-Is-All-You-Need/data_output_dec.pickle', 'rb') as datafile:
    data = pickle.load(datafile)

    current_case_number = None
    brief_id_list = list()
    brief_text_list = list()
    judgement_text = ''

    for idx, row in tqdm(data.iterrows(), total=len(list(data.iterrows()))):
        case_number = row[1]
        edms_id = row[2]
        document_type = row[4]
        text = str(row[6])

        try:
            if case_number != current_case_number:
                if len(brief_text_list) and len(judgement_text):
                    for brief_id, brief_text in zip(brief_id_list, brief_text_list):
                        data_list.append([current_case_number, brief_id, judgement_id, collapse_whitespace(cleaner(remover(brief_text))), collapse_whitespace(cleaner(remover(judgement_text)))])
                
                current_case_number = case_number
                brief_id_list = list()
                brief_text_list = list()
                judgement_text = ''

            # 01-판결문, 011-답변서, 010-준비서면, 701-소장
            if document_type == '01':
                judgement_id = edms_id
                judgement_text = text
            
            else:
                brief_id_list.append(edms_id)
                brief_text_list.append(text)
        
        except:
            print(document_type, text)
            current_case_number = case_number
            brief_id_list = list()
            brief_text_list = list()
            judgement_text = ''
            pass


with open('./Sentence-Embedding-Is-All-You-Need/abs_data.csv', 'w', encoding='utf-8-sig', newline='') as f:
    wr = csv.writer(f)

    wr.writerow(['case_number', 'brief_edms_id', 'judgement_edms_id', 'brief_text', 'judgement_text'])
    
    for data in tqdm(data_list, total=len(data_list)):
        wr.writerow(data)
